In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
colnames_origination = [
            'credit_score',
            'first_payment_date',
            'first_time_homebuyer_flag',
            'maturity_date',
            'metropolitan_area',
            'mortgage_insurance_percentage',
            'number_of_units',
            'occupancy_status',
            'original_combined_loan-to-value',
            'original_debt-to-income_ratio',
            'original_upb',
            'original_loan-to-value',
            'original_interest_rate',
            'channel',
            'prepayment_penalty_mortgage_flag',
            'amortization_type',
            'property_state',
            'property_type',
            'postal_code',
            'loan_sequence_number',
            'loan_purpose',
            'original_loan_term',
            'number_of_borrowers',
            'seller_name',
            'servicer_name',
            'super_conforming_flag',
            'pre-harp_loan_sequence_number',
            'program_indicator',
            'harp_indicator',
            'property_valuation_method',
            'interest_only_indicator'
            ]

In [3]:
colnames_performance = [
                        'loan_sequence_number',
                        'monthly_reporting_period',
                        'current_actual_upb',
                        'current_loan_delinquency_status',
                        'loan_age',
                        'remaining_months_to_legal_maturity',
                        'repurchase_flag',
                        'modification_flag',
                        'zero_balance_code',
                        'zero_balance_effective_date',
                        'current_interest_rate',
                        'current_deferred_upb',
                        'due_date_of_last_paid_installment',
                        'mi_recoveries',
                        'net_sales_proceeds',
                        'non_mi_recoveries',
                        'expenses',
                        'legal_costs',
                        'maintenance_and_preservation_costs',
                        'taxes_and_insurance',
                        'miscellaneous_expenses',
                        'actual_loss_calculation',
                        'modification_cost',
                        'step_modification_flag',
                        'deferred_payment_plan',
                        'estimated_loan-to-value',
                        'zero_balance_removal_upb',
                        'delinquent_accrued_interest',
                        'delinquency_due_to_disaster',
                        'borrower_assistance_status_code'
]

In [4]:
def read_file(file, colnames):
     return pd.read_csv(file, delimiter='|', low_memory=False, names=colnames)

In [5]:
%%time
loan_perf = pd.concat( [read_file('historical_data_time_2018Q1.txt', colnames_performance),
                                read_file('historical_data_time_2018Q2.txt', colnames_performance),
                                read_file('historical_data_time_2018Q3.txt', colnames_performance),
                                read_file('historical_data_time_2018Q4.txt', colnames_performance),
                                read_file('historical_data_time_2019Q1.txt', colnames_performance),
                                read_file('historical_data_time_2019Q2.txt', colnames_performance),
                                read_file('historical_data_time_2019Q3.txt', colnames_performance),
                                read_file('historical_data_time_2019Q4.txt', colnames_performance),
                                read_file('historical_data_time_2020Q1.txt', colnames_performance),
                                read_file('historical_data_time_2020Q2.txt', colnames_performance),
                                read_file('historical_data_time_2020Q3.txt', colnames_performance)],
                                axis=0
                                )

Wall time: 2min 39s


In [6]:
%%time
loan_orig = pd.concat( [read_file('historical_data_2018Q1.txt', colnames_origination),
                                read_file('historical_data_2018Q2.txt', colnames_origination),
                                read_file('historical_data_2018Q3.txt', colnames_origination),
                                read_file('historical_data_2018Q4.txt', colnames_origination),
                                read_file('historical_data_2019Q1.txt', colnames_origination),
                                read_file('historical_data_2019Q2.txt', colnames_origination),
                                read_file('historical_data_2019Q3.txt', colnames_origination),
                                read_file('historical_data_2019Q4.txt', colnames_origination),
                                read_file('historical_data_2020Q1.txt', colnames_origination),
                                read_file('historical_data_2020Q2.txt', colnames_origination),
                                read_file('historical_data_2020Q3.txt', colnames_origination)],
                                axis=0
                                )

Wall time: 16 s


### Selecting the columns with less than 20% missing data

In [7]:
perf_selected = []
numlines = loan_perf.shape[0]
for col in loan_perf.columns:
#     print(int(perf_18_20[col].isna().sum()*100/numlines), ' >> ', col)
    if loan_perf[col].isna().sum()*100/numlines <= 20:
        perf_selected.append(col)
    else: pass
perf_selected = perf_selected+['zero_balance_code', 'zero_balance_effective_date', 'zero_balance_removal_upb']

perf_selected

['loan_sequence_number',
 'monthly_reporting_period',
 'current_actual_upb',
 'current_loan_delinquency_status',
 'loan_age',
 'remaining_months_to_legal_maturity',
 'current_interest_rate',
 'current_deferred_upb',
 'estimated_loan-to-value',
 'zero_balance_code',
 'zero_balance_effective_date',
 'zero_balance_removal_upb']

In [8]:
orig_selected = []
numlines = loan_orig.shape[0]
for col in loan_orig.columns:
#     print(int(orig_18_20[col].isna().sum()*100/numlines), ' >> ', col)
    if loan_orig[col].isna().sum()*100/numlines <= 20:
        orig_selected.append(col)
    else: pass
    
orig_selected

['credit_score',
 'first_payment_date',
 'first_time_homebuyer_flag',
 'maturity_date',
 'metropolitan_area',
 'mortgage_insurance_percentage',
 'number_of_units',
 'occupancy_status',
 'original_combined_loan-to-value',
 'original_debt-to-income_ratio',
 'original_upb',
 'original_loan-to-value',
 'original_interest_rate',
 'channel',
 'prepayment_penalty_mortgage_flag',
 'amortization_type',
 'property_state',
 'property_type',
 'postal_code',
 'loan_sequence_number',
 'loan_purpose',
 'original_loan_term',
 'number_of_borrowers',
 'seller_name',
 'servicer_name',
 'program_indicator',
 'property_valuation_method',
 'interest_only_indicator']

In [9]:
loan_orig = loan_orig[orig_selected]
loan_perf = loan_perf[perf_selected]

## San Francisco Metro Area

In [10]:
# 41860   San Francisco-Oakland-Hayward, CA Metropolitan Statistical Area
# 36084   Oakland-Hayward-Berkeley, CA Metropolitan Division
# 41884   San Francisco-Redwood City-South San Francisco, CA Metropolitan Division
# 42034   San Rafael, CA Metropolitan Division
msa_codes_sf = [36084, 41884, 41860]

In [11]:
loan_orig_sf = loan_orig[loan_orig['metropolitan_area'].isin(msa_codes_sf)]

In [12]:
loan_data_sf = pd.merge(loan_orig_sf, loan_perf, on='loan_sequence_number')

In [13]:
loan_data_sf.to_csv('loan_perf_18_20_sf.csv', index=False)

## Miami Metro Area

In [14]:
# 22744   Fort Lauderdale-Pompano Beach-Deerfield Beach, FL Metropolitan Division
# 33124   Miami-Miami Beach-Kendall, FL Metropolitan Division
# 48424   West Palm Beach-Boca Raton-Delray Beach, FL Metropolitan Division
msa_codes_mi = [22744, 33124, 48424]

In [15]:
loan_orig_mi = loan_orig[loan_orig['metropolitan_area'].isin(msa_codes_mi)]

In [16]:
loan_data_mi = pd.merge(loan_orig_mi, loan_perf, on='loan_sequence_number')

In [17]:
loan_data_mi.to_csv('loan_perf_18_20_mi.csv', index=False)